# Live Trading Simulation Notebook
Paper trading simulation with real market data

In [ ]:
import pandas as pd
import numpy as np
import ccxt
from datetime import datetime, timedelta
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class PaperTradingSimulator:
    def __init__(self, initial_balance=10000, symbol='BTC/USDT'):
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.symbol = symbol
        self.positions = {}
        self.trades = []
        self.equity_curve = [initial_balance]
    
    def place_trade(self, side, amount, price, timestamp):
        """Simulate trade execution"""
        trade = {
            'timestamp': timestamp,
            'side': side,
            'amount': amount,
            'price': price,
            'value': amount * price,
            'fee': amount * price * 0.001  # 0.1% fee
        }
        
        self.trades.append(trade)
        
        if side == 'buy':
            cost = trade['value'] + trade['fee']
            if cost <= self.balance:
                self.balance -= cost
                if 'BTC' not in self.positions:
                    self.positions['BTC'] = 0
                self.positions['BTC'] += amount
                logger.info(f'BUY: {amount} BTC @ {price}')
        elif side == 'sell':
            if self.positions.get('BTC', 0) >= amount:
                self.balance += trade['value'] - trade['fee']
                self.positions['BTC'] -= amount
                logger.info(f'SELL: {amount} BTC @ {price}')
    
    def get_portfolio_value(self, current_price):
        btc_value = self.positions.get('BTC', 0) * current_price
        return self.balance + btc_value
    
    def get_performance_metrics(self):
        total_return = ((self.balance - self.initial_balance) / self.initial_balance) * 100
        return {'total_return': total_return, 'trades': len(self.trades)}

print('Paper Trading Simulator initialized')

In [ ]:
# Backtest with live market data
def run_simulation(strategy_signals, historical_data, initial_balance=10000):
    """Run paper trading simulation"""
    simulator = PaperTradingSimulator(initial_balance)
    
    for idx, row in historical_data.iterrows():
        if idx in strategy_signals.index:
            signal = strategy_signals[idx]
            current_price = row['close']
            
            if signal == 1 and simulator.positions.get('BTC', 0) == 0:
                amount = simulator.balance * 0.5 / current_price  # Risk 50% per trade
                simulator.place_trade('buy', amount, current_price, idx)
            elif signal == -1 and simulator.positions.get('BTC', 0) > 0:
                amount = simulator.positions['BTC']
                simulator.place_trade('sell', amount, current_price, idx)
        
        portfolio_value = simulator.get_portfolio_value(row['close'])
        simulator.equity_curve.append(portfolio_value)
    
    return simulator

print('Simulation functions defined')

In [ ]:
# Example: Run simulation with sample data
# sim_results = run_simulation(strategy_signals, df)
# metrics = sim_results.get_performance_metrics()
# print(f"Total Return: {metrics['total_return']:.2f}%")
# print(f"Total Trades: {metrics['trades']}")

print('Ready to run live trading simulation')

## Simulation Results

Track performance metrics and equity curve